In [36]:
# FUNCTION: “GET FROM INTERNET” 

import urllib.request
from bs4 import BeautifulSoup

# Funktion needs (Stock, YearGDP)

# scrape from the net:
# Beta for the stock
# dividend for the stock
# Return On Equity (In Percent)

stock = 'AAPL'
print ('looking for Beta, Dividend, and Return On Equity for the stock: ' + str(stock))
print('')

technicals = {}
try:
    url = ('http://finance.yahoo.com/q/ks?s='+stock)
    print ('using following url: ')
    print (url)
    print('')
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    
    tables = soup.findAll('table')
    for table in tables:
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')

        for row in rows:
            col_name = row.find_all('span')      # Use span to avoid supscripts
            col_name = [cell.text.strip() for cell in col_name]
            #print ('col_name: ')
            #print (col_name)
            col_val = row.find_all('td')
            col_val = [cell.text.strip() for cell in col_val]
            #print ('col_value: ')
            #print (col_val)
            technicals[col_name[0]] = col_val[1] # col_val[0] is the name cell (with subscript)

except Exception as e:
    print('Failed, exception: ', str(e))
            
#def scrape(stock_list, interested, technicals):
#    for each_stock in stock_list:
#        technicals = scrape_yahoo(each_stock)
#        print(each_stock)
        
        
interested = ['Beta (5Y Monthly)', 'Trailing Annual Dividend Rate', 'Return on Equity']
for ind in interested:
    print(ind + ": " + technicals[ind])
    print("------")

Beta = technicals['Beta (5Y Monthly)']
#print ('Beta: ' + str(Beta))
Dividend = technicals['Trailing Annual Dividend Rate']
#print ('Dividend: ' + str(Dividend))
ROE = technicals['Return on Equity']
#print ('ROE: ' + str(ROE))

looking for Beta, Dividend, and Return On Equity for the stock: AAPL

using following url: 
http://finance.yahoo.com/q/ks?s=AAPL

Beta (5Y Monthly): 1.21
------
Trailing Annual Dividend Rate: 0.82
------
Return on Equity: 103.40%
------


In [202]:
###NEUE LÖSUNG MIT einem Excel Sheet 

# scrape from the net:      
# GDP growth (real)
# Expected Inflation Rate

import pandas as pd
import numpy as np

YearGDP = 2021
print ('looking for GDP Growth and Expected Inflation Rate for year: ' + str(YearGDP))
print ('')

economicals = {}
try:
    url = ('https://ec.europa.eu/info/business-economy-euro/economic-performance-and-forecasts/economic-performance-country/euro-area_en')
    page = urllib.request.urlopen(url)
    print ('using following url: ')
    print (url)
    print ('')
    soup = BeautifulSoup(page, 'html.parser')

    # Screaping the Table Head
    tableHead = soup.findAll('th', scope = "col")
    tableHead = [cell.text.strip() for cell in tableHead]
    print ('Table Head: ')
    print (tableHead)
    print('')

    #Scraping the Table Topics
    tableTopics = soup.findAll("td", class_ = 'topic')
    tableTopics = [cell.text.strip() for cell in tableTopics]
    print ('Table Topics:')
    print (tableTopics)
    print ('')

    #Scraping the Table Content
    tableContent = soup.findAll("td", class_ = 'content')
    tableContent = [cell.text.strip() for cell in tableContent]
    print ('Table Content:')
    print (tableContent)
    print ('')
    #Dividing Table Content in Years
    print ('Dividing Table Content in Years')
    print ('length Table Head: ' + str(len(tableHead)))
    print ('')
    for n in range(len(tableHead)-1):
        print ('n: ' + str(n))
        tableContentYears[n] = [tableContent[n], tableContent[n+4], tableContent[n+8], tableContent[n+12], tableContent[n+16], tableContent[n+20]]
        print ('Table Content Year: ' + str(tableHead[n+1]))
        print (tableContentYears[n])
        print ('')

        
    ##Building Excel Sheet out of Head Topics and Content

    #Creating empty Excel File
    writer = pd.ExcelWriter('euro_area_eco_forcast.xlsx', engine='xlsxwriter')
    writer.save()

    #Opening created Excel File
    forcastSheet = pd.read_excel('euro_area_eco_forcast.xlsx')

    #Constructing Forcast Dictionary.
    print ('Adding Table Topics to Forcast Dictionary')
    forcastDict = {tableHead[0]: tableTopics}
    print ('Forcast Dictionary with Table Topics: ')
    print (forcastDict)
    print ('')
    
    for n in range(len(tableHead)-1):
        print ('Adding Table Content for Year: ' + str(tableHead[n+1]))
        forcastDict[tableHead[n+1]] = tableContentYears[n]
        print (str(forcastDict[tableHead[n+1]]))
    print ('')
    print ('Forcast Dictionary: ')
    print (forcastDict)
    print ('')
    
    
    ##Constructing DataFrame from a dictionary.
    
    print ('Constructing DataFrame from a dictionary:')
    forcastSheet = pd.DataFrame(data=forcastDict)
    print ('Forcast Sheet')
    forcastSheet

    print (forcastSheet.head())

    writer = pd.ExcelWriter('./euro_area_eco_forcast.xlsx')
    forcastSheet.to_excel(writer, 'euro_area_eco_forcast')
    writer.save()
    
except Exception as e:
    print('Failed, exception: ', str(e))
    

# German 10 Year Bond Rate (-0,25%)
#print ('looking for German Ten Year Rate for year: ' + str(YearGDP))
#print ('')
#
# check data is usable type, change type if needed.
#
# FUNCTION RETURN: (GDP Growth, ExpInflRate, Beta, Dividend, GerTenYRate, ROE)

looking for GDP Growth and Expected Inflation Rate for year: 2021

using following url: 
https://ec.europa.eu/info/business-economy-euro/economic-performance-and-forecasts/economic-performance-country/euro-area_en

Table Head: 
['Indicators', '2019', '2020', '2021', '2022']

Table Topics:
['GDP growth (%, yoy)', 'Inflation (%, yoy)', 'Unemployment (%)', 'General government balance (% of GDP)', 'Gross public debt (% of GDP)', 'Current account balance (% of GDP)']

Table Content:
['1,3', '-6,6', '4,3', '4,4', '1,2', '0,3', '1,7', '1,3', '7,5', '7,8', '8,4', '7,8', '-0,6', '-7,2', '-8,0', '-3,8', '85,8', '100,0', '102,4', '100,8', '3,1', '3,0', '3,1', '3,1']

Dividing Table Content in Years
length Table Head: 5

n: 0
Table Content Year: 2019
['1,3', '1,2', '7,5', '-0,6', '85,8', '3,1']

n: 1
Table Content Year: 2020
['-6,6', '0,3', '7,8', '-7,2', '100,0', '3,0']

n: 2
Table Content Year: 2021
['4,3', '1,7', '8,4', '-8,0', '102,4', '3,1']

n: 3
Table Content Year: 2022
['4,4', '1,3', '7,8'

In [200]:
###ALTE LÖSUNG mit drei Listen

# scrape from the net:      
# GDP growth (real)
# Expected Inflation Rate, https://ec.europa.eu/info/business-economy-euro/economic-performance-and-forecasts/economic-performance-country/euro-area_en
# German 10 Year Bond Rate (-0,25%)

YearGDP = 2021
print ('looking for GDP Growth, Expected Inflation Rate and German Ten Year Rate for year: ' + str(YearGDP))

try:
    url = ('https://ec.europa.eu/info/business-economy-euro/economic-performance-and-forecasts/economic-performance-country/eu_en')
    page = urllib.request.urlopen(url)
    print ('using following url: ')
    print (url)
    soup = BeautifulSoup(page, 'html.parser')
        
    # Scraping Table Head of Economic Forcast into a List  
    heads = soup.findAll('th')
    #print ('Heads: ')
    #print (heads)
    #print('')
    
    headList = []
    for head in heads:
        headList = headList + [head.string]
    print ('Head List: ')
    print (headList)
    print ('')
    
    # Scraping Table Topics of Economic Forcast into a List
    topics = soup.findAll('td', {"class" : 'topic'})
    #print ('Topics: ')
    #print (topics)
    #print ('')
    
    topicList = []
    for topic in topics:
        topicList = topicList + [topic.string]
    print ('Topic List: ')
    print (topicList)
    print('')
    
    # Scraping Table Content of Economic Forcast into List
    content = soup.findAll('td', {"class" : 'content'})
    #print ('Content: ')
    #print (content)
    #print ('')
    
    contentList = []
    for data in content:
        contentList = contentList + [data.string]
    print ('Content List: ')
    print (contentList)
    print ('')
    
except Exception as e:
    print('Failed, exception: ', str(e))

looking for GDP Growth, Expected Inflation Rate and German Ten Year Rate for year: 2021
using following url: 
https://ec.europa.eu/info/business-economy-euro/economic-performance-and-forecasts/economic-performance-country/eu_en
Head List: 
['Indicators', '2019', '2020', '2021', '2022']

Topic List: 
['GDP growth (%, yoy)', 'Inflation (%, yoy)', 'Unemployment (%)', 'General government balance (% of GDP)', 'Gross public debt (% of GDP)', 'Current account balance (% of GDP)']

Content List: 
['1,6', '-6,1', '4,2', '4,4', '1,4', '0,7', '1,9', '1,5', '6,7', '7,1', '7,6', '7,0', '-0,5', '-6,9', '-7,5', '-3,7', '79,1', '92,4', '94,4', '92,9', '3,0', '3,0', '3,1', '3,1']

